<font face="Calibri" size="2"> <i>SBAE - Notebook Series - Part 2, version 0.1,  June 2022. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![title](images/header.png)

# I - SBAE Time-Series Extraction & Change Detection
### Extract various time-series outputs for point data from Google Earth Engine
-------

This notebook takes you through the process of extracting outputs from various time-series change detection algorithms and structure them in a so-called data-frame (e.g. tabular structure).

### 1 - Import libs

**ONLY EXECUTE THIS CELL**

In [ ]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
import geemap
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

### 2 - Basic Input Variables

**FILL IN YOUR INPUTS**

In [ ]:
# Area of Interest

# country case example
country = 'Kenya'   
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected

# Asset case
#aoi = ee.FeatureCollection("my_aoi_feature_collection")

# Point Feature Collection from where to extract points
fc = ee.FeatureCollection("my_featurecollection_path").filterBounds(aoi)

# the column of a unique point identifier in your dataset
point_id_name = 'point_id'

grid_size = 0.25 # that's the size of the grid we are parallelizing on
workers = 10 # number of parallel EE requests

# start of calibration period (mainly for bfast)
start_calibration = "2010-01-01"

# Actual period of interest, i.e. monitoring period
start_monitor = "2015-01-01"
end_monitor =  "2020-01-01"

# Directory where output and temp files will go
outdir = None  # goes to module_results/sbae_point_analysis if left to None

# Select algorithms to run
cusum_deforest = True
bfast_monitor = True
bs_slope = True
ts_metrics = True
ccdc = False
global_products = True

# select the band to run the ts on
band='ndvi' # other choices: ndfi, ndmi, mndwi

# select the resolution to which the satellite data will be resized.
scale=30

### DO NOT CHANGE YET ###
satellite='Landsat'

### 3- Algorithm parameter settings

**Edit for advanced users, otherwise just execute**

In [ ]:
# bfast parameters
bfast_params = {
    'run': bfast_monitor,
    'start_monitor': start_monitor, 
    'freq': 365,
    'k': 3, 
    'hfrac':0.25, 
    'trend': True, 
    'level':0.05, 
    'backend':'python'}

cusum_params = {
    'run': cusum_deforest,
    'nr_of_bootstraps': 1000
}

bs_slope_params = {
    'run': bs_slope,
    'nr_of_bootstraps': 1000
}

ts_metrics_params = {
    'run': ts_metrics,
    'outlier_removal': True,
    'z_threshhold': 3
}

ccdc_params = {
    'run': ccdc,
}

global_products = {
    'run': global_products,
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir': outdir,
    'workers': workers,
    'ts_params': {
        'start_calibration': start_calibration,
        'start_monitor': start_monitor,
        'end_monitor': end_monitor,
        'point_id': point_id_name,
        'grid_size': grid_size,
        'band': band,
        'satellite': satellite,
        'scale': scale
    },
    'bfast_params': bfast_params,
    'cusum_params': cusum_params,
    'bs_slope_params': bs_slope_params,
    'ts_metrics_params': ts_metrics_params,
    'ccdc_params': ccdc_params,
    'global_products': global_products
}

# Run the whole thing

**Execute only**

In [ ]:
h.get_change_data(aoi, fc, config_dict)